In [1]:
pip install pyepo

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gurobi-machinelearning

   ---------------------------------------- 0.0/70.8 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/70.8 kB ? eta -:--:--
   ----------- ---------------------------- 20.5/70.8 kB 222.6 kB/s eta 0:00:01
   ----------------- ---------------------- 30.7/70.8 kB 262.6 kB/s eta 0:00:01
   ---------------------------------------- 70.8/70.8 kB 432.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pyepo
import torch
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import gurobi_ml


In [18]:
import time

# Iniciar temporizador para la etapa de entrenamiento
start_time_train = time.time()

In [20]:


# model for shortest path
grid = (5,5)  # grid size
optmodel = pyepo.model.grb.shortestPathModel(grid)

# generate data
num_data = 1000  # number of data
num_feat = 7  # size of feature
deg = 4  # polynomial degree
noise_width = 0.5  # noise width
x, c = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, noise_width, seed=73)

# Split the data into training and test sets
x_train, x_test, c_train, c_test = train_test_split(x, c, test_size=0.5, random_state=73)

# sklearn regressor
reg = LinearRegression()  # linear regression

# build model
twostage_model = pyepo.twostage.sklearnPred(reg)

# training
twostage_model.fit(x_train, c_train)

# Calcular el tiempo de entrenamiento
train_time = time.time() - start_time_train
print("Tiempo de entrenamiento: {:.2f} segundos".format(train_time))

# Iniciar temporizador para la etapa de predicción
start_time_predict = time.time()

# prediction
c_pred = twostage_model.predict(x_test)

# Calcular el tiempo de predicción
predict_time = time.time() - start_time_predict
print("Tiempo de predicción: {:.2f} segundos".format(predict_time))




Tiempo de entrenamiento: 0.59 segundos
Tiempo de predicción: 0.01 segundos


In [5]:

# Calculate MSE
mse = mean_squared_error(c_test, c_pred)

# Print MSE
print("MSE for Two Stage Model: ", mse)


MSE for Two Stage Model:  0.08614996931531878


In [6]:

A=optmodel.solve() # solvel

In [7]:
# Extraer la lista de soluciones del tuple
soluciones = A[0]

# Convertir la lista en un array de NumPy
soluciones_array = np.array(soluciones)

soluciones_matrix = soluciones_array.reshape(1, -1)

print("Matriz de soluciones:")
print(soluciones_matrix)

Matriz de soluciones:
[[1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [8]:
print("Dimensiones de A:", soluciones_array.shape)
print("Dimensiones de c_pred:", c_pred.shape)

Dimensiones de A: (40,)
Dimensiones de c_pred: (1000, 40)


In [10]:
costs_total_pred = np.sum(soluciones_matrix * c_pred, axis=1)


In [11]:
# Calcular el costo promedio con los c_pred
average_cost_pred = np.mean(costs_total_pred)
average_cost_pred

5.587539402052783

In [12]:
#calcular costo promrdio con los costos reales
costs_total = np.sum(soluciones_matrix * c, axis=1)


In [13]:
# Calcular el costo promedio con los c_pred
average_cost = np.mean(costs_total)
average_cost

5.537392259090267

In [14]:
Regret=(average_cost-average_cost_pred)
Regret

-0.050147142962515545

In [32]:
regret_porcentual=Regret/ average_cost 
regret_porcentual

-0.009056093665785232